The goal of this project is to be able to describe a software project which is comprised of many repos and services.

We should be able to describe: 
- All the details of each Repo or Service
- All the relationships between them

In [1]:
class Graph:
    def __init__(self):
        self.edges = set()
        self.nodes = {}
        
    def add_edge(self, edge):
        self.edges.add(edge)
        
    def add_node(self, uid, node):
        self.nodes[uid] = node

In [2]:
class Node:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return f"{self.__class__.__name__}(name='{self.name}')"
        
# class Repo(Node):
#     pass

# class Infra(Node):
#     def __init__(self, name, style):
#         super().__init__(name)
#         self.style = style

class Service(Node):
    pass



In [3]:
class Edge:
    def __init__(self, from_node_uid, to_node_uid):
        self.from_node_uid = from_node_uid
        self.to_node_uid = to_node_uid
        self.relationship = None
        self.graph = None
        
        
    def _key(self):
        return (self.relationship, self.from_node_uid, self.to_node_uid)
    
    def __hash__(self):
        return hash(self._key)
    
    def __repr__(self):
        return f"{self.__class__.__name__}(from_node_uid='{self.from_node_uid}', to_node_uid='{self.to_node_uid}')"
        
    def check_graph(self):
        if not self.graph:
            raise Exception('Edge is not part of a Graph')
        
    @property
    def from_node(self):
        self.check_graph()
        return self.graph.nodes[self.from_node_uid]
    
    @property
    def to_node(self):
        self.check_graph()
        return self.graph.nodes[self.to_node_uid]
        
        
class Calls(Edge):
    def __init__(self, caller_uid, called_uid):
        super().__init__(from_node_uid=caller_uid, to_node_uid=called_uid)
        self.relationship = 'calls'
        
    @property
    def caller(self):
        return self.from_node
    
    @property
    def called(self):
        self.to_node
        
        
# class Hosts(Edge):
#     def __init__(self, infra_uid, service_uid):
#         super().__init__(from_node_uid=infra_uid, to_node_uid=service_uid)
#         self.relationship = 'hosts'
            
#     @property
#     def infra(self):
#         return self.from_node
    
#     @property
#     def service(self):
#         return self.to_node
    
# class Deploys(Edge):
#     def __init__(self, service_uid, repo_uid):
#         super().__init__(from_node_uid=service_uid, to_node_uid=repo_uid)
#         self.relationship = 'deploys'
    
#     @property
#     def service(self):
#         return self.from_node
    
#     @property
#     def repo(self):
#         return self.to_node

So let's make the graph!

In [4]:
G = Graph()

nodes = {
#     'backend_repo': Repo('Backend'),
#     'backend_ec2': Infra('Backend', 'ec2'),
    'backend_api': Service('Backend API'),
    'websockets': Service('WebSockets'),
    'nginx': Service('NginX'),
    'route53': Service('Route53'),
    'elb': Service('ELB'),
    'frontend': Service('Frontend'),
    'osrm': Service('OSRM'),
    'app': Service('App'),
#     'backend_target_group': Infra('Backend', 'Target Group'),  
}


for uid, node in nodes.items():
    G.add_node(uid, node)
    
    
edges = [
    Calls('frontend', 'route53'),
    Calls('app', 'route53'),
    Calls('route53', 'elb'),
    Calls('elb', 'backend_api'),
    Calls('backend_api', 'websockets'),
    Calls('backend_api', 'osrm'),
    Calls('websockets', 'frontend'),
    
    
]

for edge in edges:
    G.add_edge(edge)

In [5]:
G.nodes

{'backend_api': Service(name='Backend API'),
 'websockets': Service(name='WebSockets'),
 'nginx': Service(name='NginX'),
 'route53': Service(name='Route53'),
 'elb': Service(name='ELB'),
 'frontend': Service(name='Frontend'),
 'osrm': Service(name='OSRM'),
 'app': Service(name='App')}

In [6]:
G.edges

{Calls(from_node_uid='app', to_node_uid='route53'),
 Calls(from_node_uid='backend_api', to_node_uid='osrm'),
 Calls(from_node_uid='backend_api', to_node_uid='websockets'),
 Calls(from_node_uid='elb', to_node_uid='backend_api'),
 Calls(from_node_uid='frontend', to_node_uid='route53'),
 Calls(from_node_uid='route53', to_node_uid='elb'),
 Calls(from_node_uid='websockets', to_node_uid='frontend')}